# Louvain Performance Benchmarking

This notebook benchmarks performance improvement of running the Louvain clustering algorithm within cuGraph against NetworkX. The test is run over eight test networks (graphs) and then results plotted.  
<p><p>


#### Notebook Credits

    Original Authors: Bradley Rees
    Last Edit: 06/10/2020


#### Test Environment

    RAPIDS Versions: 0.15

    Test Hardware:
    GV100 32G, CUDA 10,0
    Intel(R) Core(TM) CPU i7-7800X @ 3.50GHz
    32GB system memory



#### Updates
- moved loading ploting libraries to front so that dependencies can be checked before running algorithms
- added edge values 
- changed timing to including Graph creation for both cuGraph and NetworkX.  This will better represent end-to-end times



#### Dependencies
- RAPIDS cuDF and cuGraph version 0.6.0 
- NetworkX 
- Matplotlib 
- Scipy 
- data prep script run



#### Note: Comparison against published results


The cuGraph blog post included performance numbers that were collected over a year ago.  For the test graphs, int32 values are now used.  That improves GPUs performance.  Additionally, the initial benchamrks were measured on a P100 GPU. 

This test only comparse the modularity scores and a success is if the scores are within 15% of each other.  That comparison is done by adjusting the NetworkX modularity score and then verifying that the cuGraph score is higher.

cuGraph did a full validation of NetworkX results against cuGraph results.  That included cross-validation of every cluster.  That test is very slow and not included here

## Load the required libraries

In [ ]:
# Import needed libraries
import time
import cugraph
import cudf
import os

In [ ]:
# NetworkX libraries
import networkx as nx
from scipy.io import mmread

In [ ]:
# NetworkX libraries
try: 
    import community
except ModuleNotFoundError:
    os.system('pip install python-louvain')
    import community

In [ ]:
try: 
    import matplotlib
except ModuleNotFoundError:
    os.system('pip install matplotlib')

In [ ]:
# Loading plotting libraries
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np

### Define the test data

In [ ]:
# Test File
data = {
    'preferentialAttachment' : './data/preferentialAttachment.mtx',
    'caidaRouterLevel'       : './data/caidaRouterLevel.mtx',
    'coAuthorsDBLP'          : './data/coAuthorsDBLP.mtx',
    'dblp'                   : './data/dblp-2010.mtx',
    'citationCiteseer'       : './data/citationCiteseer.mtx',
    'coPapersDBLP'           : './data/coPapersDBLP.mtx',
    'coPapersCiteseer'       : './data/coPapersCiteseer.mtx',
    'as-Skitter'             : './data/as-Skitter.mtx'
}

### Define the testing functions

In [ ]:
# Read in a dataset in MTX format 
def read_mtx_file(mm_file):
    print('Reading ' + str(mm_file) + '...')
    M = mmread(mm_file).asfptype()
        
    return M

In [ ]:
# Run the cuGraph Louvain analytic (using nvGRAPH function)
def cugraph_call(M):

    t1 = time.time()

    # data
    gdf = cudf.DataFrame()
    gdf['src'] = M.row
    gdf['dst'] = M.col
    
    # create graph 
    G = cugraph.Graph()
    G.from_cudf_edgelist(gdf, source='src', destination='dst', renumber=False)
    
    # cugraph Louvain Call
    print('  cuGraph Solving... ')
    df, mod = cugraph.louvain(G)   
    
    t2 = time.time() - t1
    return t2, mod


In [ ]:
# Run the NetworkX Louvain analytic.  THis is done in two parts since the modularity score is not returned 
def networkx_call(M):
    nnz_per_row = {r: 0 for r in range(M.get_shape()[0])}
    for nnz in range(M.getnnz()):
        nnz_per_row[M.row[nnz]] = 1 + nnz_per_row[M.row[nnz]]
    for nnz in range(M.getnnz()):
        M.data[nnz] = 1.0/float(nnz_per_row[M.row[nnz]])

    M = M.tocsr()
    if M is None:
        raise TypeError('Could not read the input graph')
    if M.shape[0] != M.shape[1]:
        raise TypeError('Shape is not square')
        
    t1 = time.time()

    # Directed NetworkX graph
    Gnx = nx.Graph(M)

    # Networkx 
    print('  NetworkX Solving... ')
    parts = community.best_partition(Gnx)
    
    # Calculating modularity scores for comparison 
    mod = community.modularity(parts, Gnx)   
    
    t2 = time.time() - t1
    
    return t2, mod

### Run the benchmarks

In [ ]:
# Loop through each test file and compute the speedup
perf  = []
names = []
time_cu = []
time_nx = []

#init libraries by doing quick pass
v = './data/preferentialAttachment.mtx'
M = read_mtx_file(v)
trapids = cugraph_call(M)
del M


for k,v in data.items():
    M = read_mtx_file(v)
    tr, modc = cugraph_call(M)
    tn, modx = networkx_call(M)
    
    speedUp = (tn / tr)
    names.append(k)
    perf.append(speedUp)
    time_cu.append(tr)
    time_nx.append(tn)
    # mod_delta = (0.85 * modx)
    
    print(str(speedUp) + "x faster =>  cugraph " + str(tr) + " vs " + str(tn))

### plot the output

In [ ]:
%matplotlib inline

y_pos = np.arange(len(names))
 
plt.bar(y_pos, perf, align='center', alpha=0.5)
plt.xticks(y_pos, names)
plt.ylabel('Speed Up')
plt.title('Performance Speedup: cuGraph vs NetworkX')
plt.xticks(rotation=90) 
plt.show()

# Dump the raw stats

In [ ]:
perf

In [ ]:
time_cu

In [ ]:
time_nx

___
Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___